In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import torch
from pathlib import Path

from gpdm import GPDM
from mocap_labels import WALK_TRIALS, RUN_TRIALS

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


pygame 2.6.1 (SDL 2.30.7, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
walk_dfs = [trial.as_dataframe() for trial in WALK_TRIALS]
run_dfs = [trial.as_dataframe() for trial in RUN_TRIALS]

walk_arrs = [df.drop(['time', 'frame'], axis=1).to_numpy().astype(np.float32) for df in walk_dfs]
run_arrs = [df.drop(['time', 'frame'], axis=1).to_numpy().astype(np.float32) for df in run_dfs]

In [3]:
D = walk_arrs[0].shape[1]   # Observation space dimension
d = 3                       # Latent space dimension
print(D)

# Initial values for hyperparameters
y_lambdas_init = np.ones(D)  # Signal standard deviation for observation GP
y_lengthscales_init = np.ones(d)  # Lengthscales for observation GP
y_sigma_n_init = 1e-2  # Noise standard deviation for observation GP

x_lambdas_init = np.ones(d)  # Signal standard deviation for latent dynamics GP
x_lengthscales_init = np.ones(2*d)  # Lengthscales for latent dynamics GP (since dyn_back_step = 2)
x_sigma_n_init = 1e-2  # Noise standard deviation for latent dynamics GP
x_lin_coeff_init = np.ones(2*d + 1)  # Linear coefficients for latent dynamics GP


62


In [4]:
# Create GPDM object
walk_gpdm = GPDM(D=D, d=d, 
                  dyn_target='full', dyn_back_step=2, 
                  y_lambdas_init=y_lambdas_init,
                  y_lengthscales_init=y_lengthscales_init, 
                  y_sigma_n_init=y_sigma_n_init, 
                  x_lambdas_init=x_lambdas_init, 
                  x_lengthscales_init=x_lengthscales_init, 
                  x_sigma_n_init=x_sigma_n_init, 
                  x_lin_coeff_init=x_lin_coeff_init, 
                  flg_train_y_lambdas=True, 
                  flg_train_y_lengthscales=True, 
                  flg_train_y_sigma_n=True, 
                  flg_train_x_lambdas=True, 
                  flg_train_x_lengthscales=True, 
                  flg_train_x_sigma_n=True, 
                  flg_train_x_lin_coeff=True, 
                  sigma_n_num_Y=0., 
                  sigma_n_num_X=0., 
                  dtype=torch.float64, 
                  device=torch.device('cpu'))

In [5]:
config = Path().cwd() / 'walk_gpdm_config.pth'
state = Path().cwd() / 'walk_gpdm_state.pth'

walk_gpdm.load(config, state, True)


GPDM correctly loaded
Loaded params:
y_log_lengthscales 	 tensor([0.1781, 1.9632, 1.3877], dtype=torch.float64)
y_log_lambdas 	 tensor([ 2.3483,  1.6153, -0.4864, -0.6465, -1.0330, -0.7496, -1.6722, -1.2564,
        -1.5538, -1.7918, -1.8713, -1.5986, -2.2164, -1.6323, -1.4036, -1.5453,
        -1.7592, -1.8036, -1.7048, -2.0331, -0.7804, -0.0112, -0.9583, -0.2254,
        -0.3106, -0.0673, -0.7619,  0.4151, -0.3042, -1.3889, -0.2314, -0.7990,
        -1.3964, -0.5782, -0.9517, -0.7123,  0.2113, -0.0698, 10.0000, 10.0000,
        -1.6640, -1.7234, -1.6099, -1.6041, -1.7114, -1.5068, -1.5196, -0.4557,
        -1.3097, -1.5337, 10.0000, 10.0000, -1.6307, -1.5841, -1.6240, -1.6383,
        -1.6846, -1.6174, -1.6138, -0.4557, -1.6130, -1.6277],
       dtype=torch.float64)
y_log_sigma_n 	 tensor(-2.7327, dtype=torch.float64)
x_log_lengthscales 	 tensor([-0.0667,  1.4352,  1.3689,  0.8779,  1.4263,  2.1433],
       dtype=torch.float64)
x_log_lambdas 	 tensor([9.0060, 7.7296, 6.4435], dtype=

In [ ]:
# Get the log likelihood of the training data

walk_ll = walk_gpdm.log_likelihood(walk_arrs)
print(walk_ll)



In [ ]:
# Plot the latent space
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
X = walk_gpdm.X.detach().numpy()
ax.plot(X[:, 0], X[:, 1], X[:, 2])